In [1]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df=pd.read_csv('wiki_movie_plots_deduped_cleaned_genre_and_plot.csv')
df

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Genre_updated,Wiki Page,Plot,Plot_cleanned
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr...",bartender working saloon serving drink custome...
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov...",moon painted smiling face hang park night youn...
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,NaN,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed...",film minute long composed two shot first girl ...
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,NaN,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...,lasting second consisting two shot first shot ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,NaN,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...,earliest known adaptation classic fairytale fi...
...,...,...,...,...,...,...,...,...,...,...
42530,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,NaN,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ...",film begin wxyz world war ended centre around ...
42531,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the...",two musician salih gürkan described adventure ...
42532,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i...",zafer sailor living mother döndü coastal villa...
42533,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...,film centre around young woman named amy tyler...


In [3]:
data1=df[['Plot','Plot_cleanned','Genre_updated']]
data1['Word_count_of_Plot']=data1['Plot_cleanned'].apply(lambda x: len(x.split()))
data1.sort_values(by='Word_count_of_Plot',ascending=False)[:40]

<ipython-input-3-96daee713189>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Word_count_of_Plot']=data1['Plot_cleanned'].apply(lambda x: len(x.split()))


,Plot,Plot_cleanned,Genre_updated,Word_count_of_Plot
40212,After a brief introduction to some of the main...,brief introduction main character story beginn...,NaN,3577
31798,After a brief introduction to some of the main...,brief introduction main character story beginn...,NaN,3577
41679,"The film proceeds in chronological order, apar...",film proceeds chronological order apart scene ...,crime,2800
32365,"The story takes place in Pakistan in 1977, six...",story take place pakistan wxyz six year wxyz b...,war,2669
42252,The opening scene shows a group of illegal imm...,opening scene show group illegal immigrant pre...,NaN,2499
18492,Young mother Madylyn Mabry puts her daughter t...,young mother madylyn mabry put daughter bed hu...,thriller,2423
27508,Bruce Lee (Li Xiao Long) is a high school stud...,bruce lee li xiao long high school student sch...,NaN,2120
27777,The film begins in Seattle. Seattle police off...,film begin seattle seattle police officer mada...,NaN,1904
40428,"The story begins with Teru Aoki (青木 輝), the ma...",story begin teru aoki 青木 輝 main protagonist tr...,NaN,1753
27732,"When Lam Ting-yat was little, his father died ...",lam tingyat little father died raised police o...,crime,1618


In [4]:
data=pd.read_csv("most_frequent_words_300.csv")
data

,Genre,Words,Counts
0,western,town,1144
1,western,men,940
2,western,take,683
3,western,back,672
4,western,one,671
...,...,...,...
8695,anime,cause,27
8696,anime,seen,27
8697,anime,four,27
8698,anime,book,27


In [5]:
dict_words={i:(data[data['Genre']==i]['Words']).to_list() for i in data['Genre'].unique()}
dict_words

{'western': ['town',
  'men',
  'take',
  'back',
  'one',
  'kill',
  'man',
  'ranch',
  'horse',
  'find',
  'ride',
  'killed',
  'go',
  'gang',
  'return',
  'two',
  'get',
  'gun',
  'sheriff',
  'tell',
  'indian',
  'shoot',
  'father',
  'brother',
  'help',
  'try',
  'woman',
  'gold',
  'new',
  'come',
  'john',
  'gene',
  'train',
  'friend',
  'money',
  'son',
  'cattle',
  'shot',
  'make',
  'way',
  'also',
  'later',
  'escape',
  'time',
  'love',
  'away',
  'life',
  'outlaw',
  'old',
  'wife',
  'leave',
  'war',
  'see',
  'fight',
  'jim',
  'give',
  'order',
  'saloon',
  'dead',
  'wagon',
  'attack',
  'meet',
  'want',
  'billy',
  'daughter',
  'posse',
  'end',
  'however',
  'day',
  'tom',
  'home',
  'run',
  'arrives',
  'turn',
  'bill',
  'army',
  'land',
  'decides',
  'apache',
  'plan',
  'set',
  'leaf',
  'young',
  'death',
  'killing',
  'fort',
  'jack',
  'wxyz',
  'begin',
  'three',
  'night',
  'offer',
  'local',
  'year',
  'mar

In [6]:
def reduce_words(x):
    return ' '.join([i for i in x['Plot_cleanned'].split() if i in  dict_words[x['Genre_updated']]])
     

In [7]:
%%time
Train=df[~pd.isnull(df['Genre_updated'])]
Train[['Genre_updated','Plot_cleanned']][:10].apply(lambda x: reduce_words(x),axis=1)

Wall time: 18 ms


6     film two bandit railroad force train order wat...
7     film family move life thing start go wife get ...
10    leaf begin get start try onto world around man...
11                film train two make along track later
12     film train hold two make along track later horse
13            film train two rob make along track later
14      film train hold two line make along track later
15       father large causing large film family another
16    daughter start away set fire daughter return h...
17    game mr several arrives game become game back ...
dtype: object

In [8]:
%%time
Train=df[~pd.isnull(df['Genre_updated'])]
Test=df[pd.isnull(df['Genre_updated'])]


Train['Plot_trimmed']=Train[['Genre_updated','Plot_cleanned']].apply(lambda x: reduce_words(x),axis=1)


Wall time: 1min 40s


<timed exec>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [9]:
data1=Train[['Plot','Plot_cleanned','Genre_updated','Plot_trimmed']]
data1['Word_count_of_Plot']=data1['Plot_cleanned'].apply(lambda x: len(x.split()))
data1['Word_count_of_Plot_trimmed']=data1['Plot_trimmed'].apply(lambda x: len(x.split()))
data1.sort_values(by='Word_count_of_Plot_trimmed',ascending=False)[:40]

<ipython-input-9-b91bf72473e6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Word_count_of_Plot']=data1['Plot_cleanned'].apply(lambda x: len(x.split()))
<ipython-input-9-b91bf72473e6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['Word_count_of_Plot_trimmed']=data1['Plot_trimmed'].apply(lambda x: len(x.split()))


,Plot,Plot_cleanned,Genre_updated,Plot_trimmed,Word_count_of_Plot,Word_count_of_Plot_trimmed
18492,Young mother Madylyn Mabry puts her daughter t...,young mother madylyn mabry put daughter bed hu...,thriller,young mother put daughter husband jack watch a...,2423,954
32365,"The story takes place in Pakistan in 1977, six...",story take place pakistan wxyz six year wxyz b...,war,story take place wxyz year wxyz war take place...,2669,851
41679,"The film proceeds in chronological order, apar...",film proceeds chronological order apart scene ...,crime,film order scene first taken wxyz film begin w...,2800,809
28609,Kenji Delos Reyes delivers a speech during the...,kenji delos reyes delivers speech wedding rece...,teen,kenji college friend however kenneth kenji ’ s...,1210,691
28424,The film is based on selected chapters of Wu C...,film based selected chapter wu chengens classi...,martial_arts,film wu tell wukong rebel jade emperor heaven ...,1353,685
27732,"When Lam Ting-yat was little, his father died ...",lam tingyat little father died raised police o...,crime,father police officer son like brother become ...,1618,639
26915,The film is based on selected chapters of Wu C...,film based selected chapter wu chengens classi...,fantasy,film tell story wukong emperor heaven film ope...,1353,622
28423,The film is based on selected chapters of Wu C...,film based selected chapter wu chengens classi...,fantasy,film tell story wukong emperor heaven film ope...,1353,622
26390,The main objective of the film was to explain ...,main objective film explain story behind rock ...,documentary,main film story behind rock alice film start f...,1385,608
37961,A handicapped man named Krishna (Suriya) arriv...,handicapped man named krishna suriya arrives m...,action,man named krishna arrives train station train ...,1324,583


In [10]:
%%time

X_train=Train['Plot_trimmed']
y_train=Train['Genre_updated']


X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.3,random_state=123,stratify=y_train)


X_test=Test['Plot_cleanned']
y_test=Test['Genre_updated']

Wall time: 25.9 ms


In [11]:
%%time
tfidf_vectorizer = TfidfVectorizer(use_idf=True)

X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 

X_val_vectors_tfidf = tfidf_vectorizer.transform(X_val) 


X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

Wall time: 2.47 s


In [12]:
X_train_vectors_tfidf.shape,X_val_vectors_tfidf.shape,X_test_vectors_tfidf.shape,

((24984, 1525), (10708, 1525), (6843, 1525))

In [13]:
%%time
# random forest
rf = RandomForestClassifier(n_estimators=100,random_state=123,n_jobs=-1)
rf.fit(X_train_vectors_tfidf, y_train) 

Wall time: 10.4 s


RandomForestClassifier(n_jobs=-1, random_state=123)

In [14]:
#Predict y value for test dataset 
y_pred=rf.predict(X_val_vectors_tfidf)

In [15]:
print(classification_report(y_val,y_pred))
print('Confusion Matrix:',confusion_matrix(y_val, y_pred))

              precision    recall  f1-score   support

      action       0.74      0.75      0.74       742
   adventure       0.73      0.50      0.60       260
    animated       0.77      0.69      0.73       269
       anime       0.75      0.14      0.24        43
   biography       0.92      0.32      0.48       137
      comedy       0.71      0.90      0.79      2188
       crime       0.74      0.65      0.70       481
 documentary       0.67      0.05      0.10        39
       drama       0.68      0.92      0.78      2854
      family       0.72      0.20      0.31       191
     fantasy       0.70      0.31      0.43       164
  historical       0.58      0.08      0.14        86
      horror       0.84      0.84      0.84       476
martial_arts       1.00      0.33      0.49        40
     musical       0.88      0.55      0.68       288
     mystery       0.67      0.23      0.34       144
        noir       1.00      0.11      0.20       107
     romance       0.86    